In [ ]:

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier


# DataFrame
df = pd.read_csv('data.csv')

# Select all the columns except the last one as the features
X = df.iloc[:, :-1]
X = df.drop(['PRED'], axis=1)
# Select the last column as the label
y = df['PRED']

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

# Separamos nuestros datos en un conjunto de entrenamiento y uno de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=42)

# Define the distribution of hyperparameters to sample from

import scipy.stats

param_distributions = {
    'hidden_layer_sizes': [(n,) for n in range(50, 1000, 10)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': scipy.stats.loguniform(1e-5, 1e-1),
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': scipy.stats.loguniform(1e-5, 1e-1),
    'momentum': scipy.stats.uniform(0.1, 0.9),
    'beta_1': scipy.stats.uniform(0.1, 0.9),
    'beta_2': scipy.stats.uniform(0.1, 0.999),
    'epsilon': scipy.stats.uniform(1e-9, 1e-7)
}

param_distributions = {
    'n_estimators': [100, 300, 500, 800, 1000],
    'criterion': ['gini', 'entropy'],
    'max_depth': [5, 10, 15, 20, 25, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'bootstrap': [True, False],
    'class_weight': [None, 'balanced', 'balanced_subsample'],
    'min_weight_fraction_leaf': [0.0, 0.1, 0.2],
    'max_leaf_nodes': [None, 5, 10, 20, 50],
    'ccp_alpha': [0.0, 0.1, 0.2]
}

# Create a random forest classifier
clp = RandomForestClassifier(random_state=42)
clp = MLPClassifier(random_state=42)

# Create a RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=clp, param_distributions=param_distributions, 
                                   cv=10, scoring='accuracy', n_iter=10, n_jobs=-1, random_state=42)#n_iter=10

# Fit the model using SMOTE-resampled training data
random_search.fit(X_train, y_train)

# Get the best set of hyperparameters found by the random search
best_params = random_search.best_params_

# Get the best model found by the random search
best_model = random_search.best_estimator_

In [ ]:
best_model.score(X_train, y_train)

In [ ]:
best_model

In [ ]:
best_params